In [3]:
# Import and configure the sentiment analyzer.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

TWEETS_DATABASE_FILENAME = 'tweets_data.db'

In [77]:
import datetime # debuggging
import sqlite3
import os
import json

conn = sqlite3.connect(TWEETS_DATABASE_FILENAME)
conn.isolation_level = "DEFERRED"

with conn:
    cur = conn.cursor()
    print('Getting tweets...')
    cur.execute("""
    SELECT json_extract(data, '$.text'), id FROM tweets WHERE sentiment IS NULL LIMIT 50000
    """)
    results = cur.fetchall()
    cur.execute("begin")
    try:
        values_to_commit = []
        for index, result in enumerate(results):
            tweet_text = result[0]
            tweet_id = result[1]
            
            start_sentiment = datetime.datetime.now() # debugging
            sentiment_scores = analyser.polarity_scores(tweet_text)
            end_sentiment = datetime.datetime.now() # debugging
            
            values_to_commit.append([tweet_id, json.dumps(sentiment_scores), tweet_id])
            
            if index % 5000 == 0 and index != 0:
                sql = """
                    INSERT OR REPLACE INTO tweets (id, sentiment, data) 
                    VALUES (
                        ?, 
                        ?,
                        (SELECT data FROM tweets WHERE id = ?)
                    );
                """

                cur.executemany(sql, values_to_commit)
                end_update = datetime.datetime.now() # debugging

                print('updated tweet', index+1, 'of', len(results))

                values_to_commit = []
    except Error as e:
        cur.execute('rollback')
    finally:
        conn.execute('commit')

50000
updated tweet 5001 of 50000
updated tweet 10001 of 50000
updated tweet 15001 of 50000
updated tweet 20001 of 50000
updated tweet 25001 of 50000
updated tweet 30001 of 50000
updated tweet 35001 of 50000
updated tweet 40001 of 50000
updated tweet 45001 of 50000


In [4]:
import sqlite3

conn = sqlite3.connect(TWEETS_DATABASE_FILENAME)

with conn:
    cur = conn.cursor()
    print('Getting count...')
    cur.execute("SELECT COUNT(id) FROM tweets WHERE sentiment NOT NULL")
    print(cur.fetchone()[0], 'database record(s) have sentiment information')

Getting count...
314104 database record(s) have sentiment information
